    87. Design the Frontend

Ardit shows the process of him drawing the "design-frontend.jpg" file in our images folder.

The app has two screens: the Camera Screen and the Image Screen.

__Camera Screen__

The Camera Screen displays the webcam of the user. The webcam will activate when the user presses the "Start" button. The second "Capture" button will capture a frame of the active webcam and convert it to an image. At the same time, the button will open the second Image Screen and display the captured screen.

__Image Screen__

The Image Screen displays the captured image in a large widget. Below it is a large button for "Create Link". Clicking this button will activate a widget with a link for the image. Below this initially invisible widget are two different buttons: "Copy" and "Open". The Copy button will copy the image link to the user's clipboard. The Open button will open the image link in the user's browser.

    88. Designing the Object Types

Ardit simply shows the process of him creating the design.txt document.

    89. Creating the Empty Classes

Just like we learned in the previous Webcam app, we're going to introduce some boiler plate Python/Kivy code to the MainApp file.

In [ ]:
from kivy.app import App
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.lang import Builder

Builder.load_file('frontend.kv')


class CameraScreen(Screen):
    pass


class ImageScren(Screen):
    pass


class RootWidget(ScreenManager):
    pass

class MainApp(App):

    def build(self):
        return RootWidget()

MainApp().run()

Despite being listed as a class object in the design document, we are already going to define the functionality of the webcam in the CameraScreen class so we do not need to create a Webcam class of its own.